<a href="https://colab.research.google.com/github/dwdb/tensorflow2.0-tutorial/blob/master/text/text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf

import numpy as np
import os
import time

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 0s 0us/step


In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

65 unique characters


## Process the text

In [0]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [7]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'First Citizen' ---- characters mapped to int ---- > [18 47 56 57 58  1 15 47 58 47 64 43 52]


## Create training examples and targets

In [8]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

F
i
r
s
t


In [9]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=False)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [10]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [12]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


## Create training batches

In [13]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

# drop_remainder作用，是否返回长度不满足batch_size的batch

In [14]:
dataset_ = tf.data.Dataset.from_tensor_slices([[1,2],[3,4],[5,6],[7,8]])

dataset_ = dataset_.shuffle(100).batch(3, drop_remainder=True).repeat(10)

for i in dataset_.take(3):
  print(i)

tf.Tensor(
[[7 8]
 [3 4]
 [5 6]], shape=(3, 2), dtype=int32)
tf.Tensor(
[[1 2]
 [5 6]
 [3 4]], shape=(3, 2), dtype=int32)
tf.Tensor(
[[1 2]
 [3 4]
 [7 8]], shape=(3, 2), dtype=int32)


## Build The Model

In [0]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character:

![alt text](https://www.tensorflow.org/tutorials/text/images/text_generation_training.png)

## Try the model

In [18]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [20]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

sampled_indices

array([57, 38, 27, 50, 18, 44, 60,  2, 27,  7, 48, 29, 17, 23, 16, 49, 40,
       30, 22, 52, 20, 44, 24, 41, 50, 25,  9, 45,  6, 10, 16, 11, 60, 58,
        8, 64, 14, 53,  1,  4,  4, 22, 17, 32, 44, 61, 62, 21, 59, 62, 22,
       36,  7, 27, 19, 51, 34, 18, 59, 49, 54, 29, 21, 28, 38, 13, 49, 30,
       44, 16, 47, 60, 59, 16, 58,  6, 41, 61, 44, 47, 24, 19, 38, 58, 37,
       50, 21,  2,  5, 64, 44, 59, 39,  9, 18, 11, 36, 37, 40, 44])

In [21]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices])))

Input: 
 ' the commonalty.\n\nSecond Citizen:\nConsider you what services he has done for his country?\n\nFirst Cit'

Next Char Predictions: 
 "sZOlFfv!O-jQEKDkbRJnHfLclM3g,:D;vt.zBo &&JETfwxIuxJX-OGmVFukpQIPZAkRfDivuDt,cwfiLGZtYlI!'zfua3F;XYbf"


In [22]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 65)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.173262


In [0]:
model.compile(optimizer='adam', loss=loss)


In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [25]:
EPOCHS=10

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 7s 41ms/step - loss: 2.6684
Epoch 2/10
172/172 [==============================] - 7s 41ms/step - loss: 1.9649
Epoch 3/10
172/172 [==============================] - 7s 41ms/step - loss: 1.6959
Epoch 4/10
172/172 [==============================] - 7s 41ms/step - loss: 1.5467
Epoch 5/10
172/172 [==============================] - 7s 41ms/step - loss: 1.4582
Epoch 6/10
172/172 [==============================] - 7s 41ms/step - loss: 1.3990
Epoch 7/10
172/172 [==============================] - 7s 42ms/step - loss: 1.3537
Epoch 8/10
172/172 [==============================] - 7s 41ms/step - loss: 1.3146
Epoch 9/10
172/172 [==============================] - 7s 41ms/step - loss: 1.2794
Epoch 10/10
172/172 [==============================] - 7s 42ms/step - loss: 1.2480


In [26]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_10'

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [28]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 65)             66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


## The prediction loop
The following code block generates the text:

- It Starts by choosing a start string, initializing the RNN state and setting the number of characters to generate.

- Get the prediction distribution of the next character using the start string and the RNN state.

- Then, use a categorical distribution to calculate the index of the predicted character. Use this predicted character as our next input to the model.

- The RNN state returned by the model is fed back into the model so that it now has more context, instead than only one character. After predicting the next character, the modified RNN states are again fed back into the model, which is how it learns as it gets more context from the previously predicted characters.

![alt text](https://www.tensorflow.org/tutorials/text/images/text_generation_sampling.png)

In [0]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      # 缩放logits，temperature越大，输出结果越随机
      predictions = predictions / temperature
      
      # shape=(batch_size, num_samples), 每个样本采样num_samples个候选输出
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [52]:
print(generate_text(model, start_string=u"ROMEO: "))

ROMEO: Camilloking fortune's together,
And their woes have very little though, but witt thy temper:
hold mine eyes, sweet Warwick Musicians,
Then have the nobles ig you will
consider of me.

HORTENSIO:
How shath a thousand to his friends the town;
The ree cricket ridst a raib.

FLORIZEL:
No, it is a brinces to thine inr, I may know
The duke-you Lord Angelo, why is it light:
I mean, my lord, between your spittle, ears.

MOPSA:
Sir, it who is not sin to--
To thrive away to thee to sure our foot.

KING RITHARD III:
Bishop by him to thy leadned thee.
Farerel, for I had rather to thy duty--but thousands;
For tenterman is full of mine own good with thick to inst thou living prince:
The peerly,
Make me with me.

LUCIO:
Nine Bianca say: the more indeed will feel
a burgen Ghell; Nay, he comes himself.
I am no power to deposed;
And I beseech you then: if he be hath waken.

QUEEN MARBARET:
Thy gods, which is thy horse and here!

PETRUCHIO:
'Tis dropp roundly when his crown?

GLOUCESTER:
I cannot 

## Advanced: Customized Training

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [0]:
optimizer = tf.keras.optimizers.Adam()

In [0]:
@tf.function
def train_step(inp, target):
  with tf.GradientTape() as tape:
    predictions = model(inp)
    loss = tf.reduce_mean(
        tf.keras.losses.sparse_categorical_crossentropy(
            target, predictions, from_logits=True))
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  return loss

In [56]:
# Training step
EPOCHS = 10

for epoch in range(EPOCHS):
  start = time.time()

  # initializing the hidden state at the start of every epoch
  # initally hidden is None
  hidden = model.reset_states()

  for (batch_n, (inp, target)) in enumerate(dataset):
    loss = train_step(inp, target)

    if batch_n % 100 == 0:
      template = 'Epoch {} Batch {} Loss {}'
      print(template.format(epoch+1, batch_n, loss))

  # saving (checkpoint) the model every 5 epochs
  if (epoch + 1) % 5 == 0:
    model.save_weights(checkpoint_prefix.format(epoch=epoch))

  print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
  print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 4.1732330322265625
Epoch 1 Batch 100 Loss 2.3562841415405273
Epoch 1 Loss 2.1361
Time taken for 1 epoch 8.318379163742065 sec

Epoch 2 Batch 0 Loss 2.1341233253479004
Epoch 2 Batch 100 Loss 1.9271557331085205
Epoch 2 Loss 1.8353
Time taken for 1 epoch 7.320577144622803 sec

Epoch 3 Batch 0 Loss 1.766493558883667
Epoch 3 Batch 100 Loss 1.6475504636764526
Epoch 3 Loss 1.6254
Time taken for 1 epoch 7.356818199157715 sec

Epoch 4 Batch 0 Loss 1.6349270343780518
Epoch 4 Batch 100 Loss 1.551010012626648
Epoch 4 Loss 1.4897
Time taken for 1 epoch 7.402777433395386 sec

Epoch 5 Batch 0 Loss 1.4808212518692017
Epoch 5 Batch 100 Loss 1.4703172445297241
Epoch 5 Loss 1.4158
Time taken for 1 epoch 7.380358457565308 sec

Epoch 6 Batch 0 Loss 1.3682001829147339
Epoch 6 Batch 100 Loss 1.3896385431289673
Epoch 6 Loss 1.4341
Time taken for 1 epoch 7.405625820159912 sec

Epoch 7 Batch 0 Loss 1.3454155921936035
Epoch 7 Batch 100 Loss 1.3354101181030273
Epoch 7 Loss 1.3599
Time taken f